In [ ]:
# Written in python2.
# This script renders a webpage whenever a CAPTCHA challenge is encountered, which needs to be solved by the user.
# url and filename need to be changed if looking for public companies.
# Sleep delays should be adjusted as required.
# Choose a small list of initials at a time.

In [1]:
from selenium import webdriver 
# import winsound
import unidecode
import time
from HTMLParser import HTMLParser

In [2]:
options = webdriver.ChromeOptions()
options.binary_location = "/Users/jiayi666/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome"
chrome_driver_path = "/Users/jiayi666/Downloads/chromedriver"
# browser = webdriver.Chrome(chrome_driver_path, chrome_options=options)
browser = webdriver.Chrome(chrome_driver_path)

In [3]:
# initials = [chr(j) for j in range(65,91)]+[chr(i) for i in range(48,58)]
initials = ["A"]
print initials

['A']


In [4]:
def all_companies_in_page(x):
    page_list=[]
    table = x.find('"table"')
    start = x.find('<tbody>',table)
    end = x.find('</tbody>',table)
    flag = 1
    k = start
    while k<end and flag:
        if x.find('">',k,end):
            k = x.find('">',k)
            try:
                clast = x.index('</a>',k+2,end)                
                company = x[k+2:clast]
                k =clast
                country_beg = x.index('<td>',clast)
                country_end = x.index('</td>',country_beg)                
                country = x[country_beg+4:country_end]
                page_list+=[(company,country)]
            except:
                break
        else:
            k += 1
    return page_list

In [5]:
def check_iscaptcha(x):
    if 'Are you a robot?' in x:
        winsound.Beep(300, 2000)
        while(1):
            try:
                input("Have you solved the captcha? Enter a number to proceed. ")
                return True
            except NameError:
                print ("Don't forget the quotes if entering a character!")
                pass
    else:
        return False

In [6]:
def all_pages_in_initial(initial):
    companies_in_initial = []   

    url = 'https://www.bloomberg.com/research//common/symbollookup/symbollookup.asp?lookuptype=private&region=all&letterIn='+initial
    print url   
    
    browser.get(url)
    x = HTMLParser().unescape(unidecode.unidecode(unicode(browser.page_source)))
    if check_iscaptcha(x):
        browser.get(url)
        x = HTMLParser().unescape(unidecode.unidecode(unicode(browser.page_source)))
    
    companies_in_initial+= all_companies_in_page(x)
    f= x.index('returned')
    j = f+1
    num = ''
    flag = 0
    while j:
        if x[j].isdigit():
            num+=x[j]
            flag = 1
        if x[j]==' ' and flag == 1:
            break                        
        j += 1
    num = int(num)
    print 'No of search results in ',initial,' = ',num
    for g in range(180,num,180):
        
        t = time.time()
        url = 'www.bloomberg.com/research//common/symbollookup/symbollookup.asp?lookuptype=private&region=all&letterIn='+initial+'&firstrow='+str(g)
       
        print url
        time.sleep(2)
        
        browser.get(url)
        x = HTMLParser().unescape(unidecode.unidecode(unicode(browser.page_source)))
        if check_iscaptcha(x):
            browser.get(url)
            x = HTMLParser().unescape(unidecode.unidecode(unicode(browser.page_source)))
        
        if time.time()-t >20:
            winsound.Beep(1000,1000)
            browser.get(url)
            x = HTMLParser().unescape(unidecode.unidecode(unicode(browser.page_source)))
            
        
        companies_in_initial+= all_companies_in_page(x)
    
    print 'No of companies in initial ',initial,'= ',len((companies_in_initial))

    with open('Data/Company_lists/bloomberg_private.txt','a') as g:
        for pair in companies_in_initial:
            g.write(pair[0].encode('utf8')+','.encode('utf8')+pair[1].encode('utf8')+'\n'.encode('utf-8'))
            
    time.sleep(2)
    
            

In [7]:
for initial in initials:
    all_pages_in_initial(initial)

https://www.bloomberg.com/research//common/symbollookup/symbollookup.asp?lookuptype=private&region=all&letterIn=A
No of search results in  A  =  95687
www.bloomberg.com/research//common/symbollookup/symbollookup.asp?lookuptype=private&region=all&letterIn=A&firstrow=180


WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"Cannot navigate to invalid URL"}
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Mac OS X 10.12.6 x86_64)
